In [1]:
# Import
from datacore.models import Concept
import networkx as nx
import matplotlib.pyplot as plt

In [77]:
graph_attribute_color = {
    "ADJ": "#1ECBE1",
    "v": "#E1341E",
    "r": "#9BDD22",
    "NOUN": "#5b6d7c",
    "s": "#F1AB41",
}
graph_edge_color = {  # relationship_choices
    "ADJ": "#7C6B94",
    "n": "#946b6e",
    "x": "#83946b",
    "y": "#6b9491",
    "u": "#a956a1",
    "g": "#a98856",
    "a": "#56a95e",
    "f": "#5677a9",
    "c": "#c1403e",
    "e": "#7dc13e",
    "l": "#3ebfc1",
    "o": "#823ec1",
    "p": "#FF5500",
    "m": "#e21db6",
    "s": "#e2ac1d",
    "i": "#1de249",
    "r": "#1d53e2",
    "d": "#ff0000",
    "v": "#80ff00",
}

In [80]:
def draw_graph(graph, path="", name="graph.png"):
    # for concept in concepts:
    #    graph.nodes[concept.id]['attribute'] = concept.attribute
    #    graph.nodes[concept.id]['color'] = graph_attribute_color[concept.attribute]
    # f = plt.figure(figsize=[12, 10])
    f = plt.figure(figsize=[6 + len(graph.nodes), 4 + len(graph.nodes)])
    pos = nx.spring_layout(graph, k=1, iterations=20)
    limits = plt.axis("off")
    for node in graph.nodes:
        # print("~~~~~~~~~~~~~~")
        # print(node)
        # print(graph.nodes[node])
        # print(graph.nodes[node]['attribute'])
        graph.nodes[node]["color"] = graph_attribute_color[
            graph.nodes[node]["attribute"]
        ]
    # print(graph.nodes.data(data=True))
    # print(graph.edges.data(data=True))

    nx.draw_networkx_labels(
        graph,
        pos=pos,
        font_size=9,
        labels={key: value for (key, value) in graph.nodes.data("title")},
    )
    for key in graph_edge_color:
        nx.draw_networkx_edges(
            graph,
            pos=pos,
            edge_color=graph_edge_color[key],
            edgelist=list(
                [x[0], x[1]] for x in graph.edges.data() if x[2]["type"] == key
            ),
        )

    nx.draw_networkx_edge_labels(
        graph,
        pos=pos,
        font_size=9,
        edge_labels={(x, y): value for (x, y, value) in graph.edges.data("title")},
    )
    for key in graph_attribute_color:
        nx.draw_networkx_nodes(
            graph,
            pos=pos,
            edgecolors="#AAAAAA",
            node_color=graph_attribute_color[key],
            nodelist=list(
                x[0]
                for x in graph.nodes().data()
                if x[1]["color"] == graph_attribute_color[key]
            ),
        )

    # list(x for x in d if x[1]['color'] == '#1ECBE1')
    # print(color)
    import os

    print(os.path.join(path, name))
    f.savefig(os.path.join(path, name))

    # Close figure and plot to release memory
    f.clf()
    plt.close("all")
    # plt.close()


"""
----------------
    Concept
----------------
"""


def get_direct_relations(id):
    relations = Relation.objects.filter(concepts__contains=[id])
    return relations


def get_direct_relations_graph(id):
    graph = nx.MultiDiGraph()
    relations = get_direct_relations(id)  # add all nodes and edges(id only)

    for rel in relations:
        x, y = (
            (rel.concepts[0], rel.concepts[1])
            if rel.concepts[0] == id
            else (rel.concepts[1], rel.concepts[0])
        )
        graph.add_edge(x, y, type=rel.relation_type, title=rel.relation_type)
    concepts = list(
        Concept.objects.filter(id__in=[key for key, value in graph.nodes.data()])
    )  # query
    for concept in concepts:
        graph.nodes[concept.id]["title"] = concept.get_title()
        graph.nodes[concept.id]["attribute"] = concept.pos
    # print(graph.nodes.data(data=True))
    # print(graph.edges.data(data=True))
    return graph


def get_hierarchy_relations_graph(id):
    graph = nx.MultiDiGraph()
    pkids = [id]
    newpkids = []
    ongoing = True
    while ongoing:
        if pkids:
            for pkid in pkids:
                relations = get_direct_relations(id)
                ongoing = False
                for rel in relations:
                    x, y = (
                        (rel.concepts[0], rel.concepts[1])
                        if rel.concepts[0] == id
                        else (rel.concepts[1], rel.concepts[0])
                    )
                    if (
                        rel.relation_type == "HYPONYM" and pkid == x
                    ):  #  and pkid == rel.relation["x"]
                        graph.add_edge(
                            x, y, type=rel.relation_type, title=rel.relation_type
                        )
                        newpkids.append(y)
                        ongoing = True
            pkids = newpkids
            newpkids = []

    concepts = list(
        Concept.objects.filter(id__in=[key for key, value in graph.nodes.data()])
    )  # query
    for concept in concepts:
        graph.nodes[concept.id]["title"] = concept.get_title()
        graph.nodes[concept.id]["attribute"] = concept.pos
    return graph


def get_neighborhood_relations_graph(id, n=1):
    graph = nx.MultiDiGraph()
    pkids = []
    pkids.append(id)
    newpkids = []
    oldpkids = []
    while n > 0:
        if pkids:
            for pkid in pkids:
                if pkid not in oldpkids:
                    oldpkids.append(pkid)
                    newpkids = []
                    concept = Concept.objects.get(id=pkid)
                    relations = get_direct_relations(id=pkid)
                    # print(relations)
                    for rel in relations:
                        x, y = (
                            (rel.concepts[0], rel.concepts[1])
                            if rel.concepts[0] == id
                            else (rel.concepts[1], rel.concepts[0])
                        )
                        graph.add_edge(
                            x, y, type=rel.relation_type, title=rel.relation_type
                        )
                        newpkids.append(x) if x not in oldpkids else None
                        newpkids.append(y) if y not in oldpkids else None

        pkids = newpkids
        n = n - 1

    concepts = list(
        Concept.objects.filter(id__in=[key for key, value in graph.nodes.data()])
    )  # query
    for concept in concepts:
        graph.nodes[concept.id]["title"] = concept.get_title()
        graph.nodes[concept.id]["attribute"] = concept.pos
    return graph


def draw_concept_graphs(id):
    from project.settings import MEDIA_ROOT

    graph = get_direct_relations_graph(id=id)
    draw_graph(graph, MEDIA_ROOT, "{}-{}.png".format(id, "relation"))
    graph = get_hierarchy_relations_graph(id=id)
    draw_graph(graph, MEDIA_ROOT, "{}-{}.png".format(id, "hierarchy"))
    graph = get_neighborhood_relations_graph(id, n=2)
    draw_graph(graph, path=MEDIA_ROOT, name="{}-{}.png".format(id, "neighborhood"))

In [81]:
draw_concept_graphs(118341)

C:\prj\elevate\project\project\media\118341-relation.png
C:\prj\elevate\project\project\media\118341-hierarchy.png
C:\prj\elevate\project\project\media\118341-neighborhood.png
